# Sleep Habits Data Prep

In [25]:
import datetime as dt
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.core.interactiveshell import InteractiveShell
import synapseclient
from synapseclient import Activity, Schema, Table, as_table_columns

InteractiveShell.ast_node_interactivity = 'all'
syn = synapseclient.Synapse()
syn.login()


UPGRADE AVAILABLE

A more recent version of the Synapse Client (2.0.0) is available. Your version (1.9.4) can be upgraded by typing:
    pip install --upgrade synapseclient

Python Synapse Client version 2.0.0 release notes

https://python-docs.synapse.org/build/html/news.html



Welcome, Sean Deering!



### Get Raw Sleep Habits Data

In [26]:
sleephabits_raw_id = 'syn7117919'
sleephabits = pd.read_csv(syn.get(sleephabits_raw_id).path)

## Replace white space with NaN

In [27]:
sleephabits = sleephabits.replace(r'^\s*$', np.nan, regex=True)

### Process Sleep Habits Data

In [28]:
#convert seconds to hours   
sleephabits.sleep_time_weekday = sleephabits.sleep_time_weekday.astype(float)/3600
sleephabits.sleep_time_weekend = sleephabits.sleep_time_weekend.astype(float)/3600
sleephabits.sleep_time_weekday = sleephabits.sleep_time_weekday.round(decimals=2)
sleephabits.sleep_time_weekend = sleephabits.sleep_time_weekend.round(decimals=2)

#rename sleep_time_weekday to sleep_time_workday so it's more consistent with the way the question was asked
sleephabits = sleephabits.rename(columns={'sleep_time_weekday':'sleep_time_workday'})

#convert seconds to minutes
sleephabits.sleep_lost = sleephabits.sleep_lost.astype(float)/60
sleephabits.sleep_lost = sleephabits.sleep_lost.round(decimals=2)

#remove brackets
def remove_brackets(col):
    return( col.astype(str).str.replace(']', '').str.replace('[',''))
sleephabits.alarm_dependency = remove_brackets(sleephabits.alarm_dependency)
sleephabits.driving_sleepy = remove_brackets(sleephabits.driving_sleepy)
sleephabits.falling_asleep = remove_brackets(sleephabits.falling_asleep)
sleephabits.morning_person = remove_brackets(sleephabits.morning_person)
sleephabits.nap_duration = remove_brackets(sleephabits.nap_duration)
sleephabits.sleep_partner = remove_brackets(sleephabits.sleep_partner)
sleephabits.wake_up_choices = remove_brackets(sleephabits.wake_up_choices)
sleephabits.weekly_naps = remove_brackets(sleephabits.weekly_naps)
sleephabits.what_wakes_you = remove_brackets(sleephabits.what_wakes_you)

### Filter based on age

In [29]:
underage_participants = pd.read_csv( syn.get('syn21905452').path, sep="\t")
sleephabits = sleephabits[~sleephabits.participantId.isin(underage_participants.participantId)]

### Clean Sleep Habits data

### Clean wake_ups

In [30]:
sleephabits.wake_ups.describe()
sleephabits.wake_ups.quantile([.01, .05, .5, .6, .8, .85, .90, .95, .97, 0.99, 0.999, 1])

count    5404.000000
mean        2.207624
std         2.866357
min         0.000000
25%         1.000000
50%         2.000000
75%         3.000000
max       100.000000
Name: wake_ups, dtype: float64

0.010      0.0
0.050      0.0
0.500      2.0
0.600      2.0
0.800      3.0
0.850      4.0
0.900      4.0
0.950      5.0
0.970      6.0
0.990     10.0
0.999     25.0
1.000    100.0
Name: wake_ups, dtype: float64

In [31]:
## Exclude if wake ups is > .99 percentile of the sample in this case > 10
sleephabits.wake_ups[ sleephabits.wake_ups > 10 ] = 'LOW DATA QUALITY'
sleephabits.wake_ups.value_counts()

1.0                 1404
2.0                 1385
3.0                  907
0.0                  860
4.0                  325
5.0                  291
6.0                   80
10.0                  44
7.0                   42
LOW DATA QUALITY      34
8.0                   28
9.0                    4
Name: wake_ups, dtype: int64

### Clean sleep_needed

In [32]:
sleephabits.sleep_needed.describe()
sleephabits.sleep_needed.quantile([.01, .05, .5, .6, .8, .85, .90, .95, .97, 0.99, 0.999, 1])

count    5356.000000
mean        7.553211
std         1.362142
min         0.000000
25%         7.000000
50%         8.000000
75%         8.000000
max        24.000000
Name: sleep_needed, dtype: float64

0.010     4.0
0.050     5.0
0.500     8.0
0.600     8.0
0.800     8.0
0.850     9.0
0.900     9.0
0.950    10.0
0.970    10.0
0.990    11.0
0.999    14.0
1.000    24.0
Name: sleep_needed, dtype: float64

In [33]:
## Exclude if sleep needed is > .99 percentile of the sample in this case > 11.43
sleephabits.sleep_needed[ sleephabits.sleep_needed > 11.43 ] = 'LOW DATA QUALITY'
sleephabits.sleep_needed.value_counts()

8.0                 2026
7.0                 1506
9.0                  616
6.0                  613
10.0                 231
5.0                  194
4.0                   72
LOW DATA QUALITY      52
11.0                  19
3.0                    8
2.0                    8
1.0                    7
0.0                    4
Name: sleep_needed, dtype: int64

### Clean sleep_lost

In [34]:
sleephabits.sleep_lost.describe()
sleephabits.sleep_lost.quantile([.01, .05, .5, .6, .8, .85, .90, .95, .97, 0.99, 0.999, 1])

count    2952.000000
mean       22.651423
std        55.621458
min         1.000000
25%         1.000000
50%         1.000000
75%        22.000000
max      1439.000000
Name: sleep_lost, dtype: float64

0.010       1.000
0.050       1.000
0.500       1.000
0.600      10.000
0.800      30.000
0.850      60.000
0.900      60.000
0.950     120.000
0.970     121.000
0.990     181.000
0.999     605.929
1.000    1439.000
Name: sleep_lost, dtype: float64

In [35]:
## Exclude if sleep lost is > .99 percentile of the sample in this case > 181
sleephabits.sleep_lost[ sleephabits.sleep_lost > 181 ] = 'LOW DATA QUALITY'
sleephabits.sleep_lost.value_counts()

1.0                 1580
30.0                 200
60.0                 186
10.0                 169
15.0                 161
20.0                 123
5.0                  115
61.0                  91
120.0                 56
121.0                 38
LOW DATA QUALITY      29
45.0                  27
180.0                 24
90.0                  19
3.0                   16
25.0                  12
181.0                 10
6.0                   10
2.0                    8
40.0                   8
4.0                    7
7.0                    5
12.0                   5
8.0                    4
13.0                   3
150.0                  3
18.0                   3
75.0                   3
22.0                   2
28.0                   2
34.0                   2
32.0                   2
66.0                   2
65.0                   2
35.0                   2
62.0                   2
46.0                   2
31.0                   1
42.0                   1
68.0                   1


### Clean sleep_time_workday

In [36]:
sleephabits.sleep_time_workday.describe()
sleephabits.sleep_time_workday.quantile([.01, .05, .5, .6, .8, .85, .90, .95, .97, 0.99, 0.999, 1])

count    5431.000000
mean        6.641291
std         1.526127
min         0.020000
25%         6.000000
50%         7.000000
75%         7.500000
max        17.000000
Name: sleep_time_workday, dtype: float64

0.010     0.02
0.050     4.33
0.500     7.00
0.600     7.00
0.800     7.90
0.850     8.00
0.900     8.00
0.950     8.50
0.970     9.00
0.990    10.00
0.999    12.02
1.000    17.00
Name: sleep_time_workday, dtype: float64

In [37]:
## Exclude if sleep time workday is > .99 percentile of the sample in this case > 10
sleephabits.sleep_time_workday[ sleephabits.sleep_time_workday > 10 ] = 'LOW DATA QUALITY'
sleephabits.sleep_time_workday.value_counts()

7.0                 1089
6.0                  761
8.0                  554
6.5                  355
7.5                  346
7.02                 310
5.0                  299
6.02                 227
8.02                 158
9.0                  128
5.5                  125
5.02                 105
4.0                   86
0.02                  70
8.5                   67
6.75                  54
10.0                  45
7.25                  43
9.02                  35
6.25                  33
LOW DATA QUALITY      31
4.5                   30
4.02                  26
7.33                  24
6.67                  19
5.75                  17
7.75                  17
3.0                   16
7.17                  15
9.5                   11
                    ... 
7.78                   1
3.73                   1
5.27                   1
6.2                    1
6.1                    1
6.22                   1
4.08                   1
9.23                   1
8.8                    1


### Clean sleep_time_weekend

In [39]:
sleephabits.sleep_time_weekend = sleephabits['sleep_time_weekend']

sleephabits.sleep_time_weekend.quantile([.01, .05, .5, .6, .8, .85, .90, .95, .97, 0.99, 0.999, 1])
sleephabits.sleep_time_weekend.describe()

0.010     0.02000
0.050     5.00000
0.500     8.00000
0.600     8.02000
0.800     9.00000
0.850     9.02000
0.900    10.00000
0.950    10.02000
0.970    11.02000
0.990    12.02000
0.999    16.01136
1.000    20.02000
Name: sleep_time_weekend, dtype: float64

count    5433.000000
mean        7.828776
std         1.858580
min         0.020000
25%         7.000000
50%         8.000000
75%         9.000000
max        20.020000
Name: sleep_time_weekend, dtype: float64

In [40]:
## Exclude if sleep time weekend is > .99 percentile of the sample in this case > 12.02 
sleephabits.sleep_time_weekend[ sleephabits.sleep_time_weekend > 12.02 ] = 'LOW DATA QUALITY'
sleephabits.sleep_time_weekend.value_counts()

[WARNING] /usr/local/lib/python2.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  



8.0                 1052
7.0                  668
9.0                  653
8.02                 337
10.0                 310
6.0                  283
7.02                 252
9.02                 231
7.5                  211
8.5                  172
6.02                 142
10.02                138
5.0                  105
6.5                   81
12.0                  57
0.02                  57
11.0                  56
5.02                  56
12.02                 43
4.0                   36
9.5                   32
11.02                 32
LOW DATA QUALITY      28
5.5                   23
8.25                  20
7.75                  19
3.0                   14
7.25                  13
4.02                  13
4.5                   13
                    ... 
0.05                   1
8.15                   1
8.47                   1
8.55                   1
6.7                    1
8.87                   1
7.8                    1
0.07                   1
7.35                   1


### Create Internal & External Copies of the Data

In [41]:
#download the data 
sharing_info = pd.read_excel(syn.get('syn21557215').path)
healthCodes_with_broadsharing = sharing_info[sharing_info.sharing == 'all_qualified_researchers']

EXTERNAL_SLEEPHABITS_DATA = sleephabits[sleephabits.participantId.isin(healthCodes_with_broadsharing['participant id'])]
EXTERNAL_SLEEPHABITS_DATA.shape

(3248, 16)

### External - Upload to Synapse

In [ ]:
SH_EXTERNAL_PROJECT = 'syn18492837'
table_schema_external = Schema(name='Sleep Habits',
                               columns=as_table_columns(EXTERNAL_SLEEPHABITS_DATA),
                               parent=SH_EXTERNAL_PROJECT)
sleephabits_synTable_external = syn.store(Table(table_schema_external,EXTERNAL_SLEEPHABITS_DATA))
sleephabits_synTable_external

### Internal - Upload to Synapse

In [ ]:
SH_INTERNAL_PROJECT = 'syn7066726'
table_schema_internal = Schema(name='Sleep Habits Internal',
                               columns=as_table_columns(sleephabits), 
                               parent=SH_INTERNAL_PROJECT)
sleephabits_synTable_internal = syn.store(Table(table_schema_internal,sleephabits))

### Set Provenance

In [ ]:
activity=Activity(name= 'Sleep Habits data curation', 
                  description='Process and convert raw data to table format', 
                  used=sleephabits_raw_id, 
                  executed='https://github.com/apratap/SleepHealth_Data_Release/blob/master/Create_Sleep_Habits.ipynb')
#Internal 
syn.setProvenance(sleephabits_synTable_internal, activity)

#External
syn.setProvenance(sleephabits_synTable_external, activity)